# Kerasを用いた画像分類入門（２）

##  過学習の抑制

## Google Colab用の設定

Google Colaboratory で実行する場合には下記のセルを実行してください。

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!unzip /content/gdrive/My\ Drive/GD_share02/noodles.zip -d /content

## ライブラリのインポート

In [ ]:
import tensorflow.keras as keras

## 画像データの準備

 解凍した教師画像からデータを読み込むImageDataGeneratorを作成します。

In [ ]:
train_dir = '/content/noodles/train/'
validation_dir = '/content/noodles/validation/'
test_dir = '/content/noodles/test/'
image_size = (128, 128)
batch_size = 32
steps_per_epoch = 300 // batch_size
validation_steps = 100 // batch_size
test_steps = 100 // batch_size

image_data_generator = keras.preprocessing.image.ImageDataGenerator(rescale=1/255.0)

train_generator = image_data_generator.flow_from_directory(train_dir,
                                                           target_size=image_size, 
                                                           batch_size=batch_size)
validation_generator = image_data_generator.flow_from_directory(validation_dir,
                                                                target_size=image_size, 
                                                                batch_size=batch_size)
test_generator = image_data_generator.flow_from_directory(test_dir,
                                                                target_size=image_size, 
                                                                batch_size=batch_size)

## ニューラルネットワークの過学習とは？

###  モデルの構築

Conv2DとMaxPooling2Dを用いた画像の２値分類のための畳み込みニューラルネットワークを構成します。

In [ ]:
model = keras.models.Sequential()

model.add(keras.layers.Conv2D(32, (3, 3), strides=(1, 1), activation='relu', input_shape=(128, 128, 3)))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Conv2D(64, (3, 3), strides=(1, 1), activation='relu'))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Conv2D(128, (3, 3), strides=(1, 1), activation='relu'))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dense(2, activation='softmax'))

### モデル構造の表示

In [ ]:
model.summary()

### モデルのコンパイル

モデルをコンパイルします。

In [ ]:
model.compile(loss='categorical_crossentropy', 
              optimizer='adam',
              metrics=['acc'])

### モデルの訓練

ここでは20エポック、モデルの訓練を行います。

In [ ]:
history = model.fit_generator(train_generator,
                              epochs=20,
                              steps_per_epoch=steps_per_epoch,
                              validation_data=validation_generator,
                              validation_steps=validation_steps)

### 学習経過のグラフ化

下記のプログラムで、エポックごとの損失と正解率をグラフ化できます。  
グラフから何が読み取れるでしょうか？

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, label='Train Accuracy')
plt.plot(epochs, val_acc, label='Validation Accuracy')
plt.legend()
plt.show()

plt.plot(epochs, loss, label='Train Loss')
plt.plot(epochs, val_loss, label='Validation Loss')
plt.legend()
plt.show()

### モデルの評価

In [ ]:
evaluation = model.evaluate_generator(test_generator, steps=test_steps)
print(evaluation)

## ドロップアウトの導入

### モデルの修正

最初のモデルにDropout層を挿入してみましょう。

In [ ]:
model = keras.models.Sequential()

model.add(keras.layers.Conv2D(32, (3, 3), strides=(1, 1), activation='relu', input_shape=(128, 128, 3)))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Conv2D(64, (3, 3), strides=(1, 1), activation='relu'))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Conv2D(128, (3, 3), strides=(1, 1), activation='relu'))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(2, activation='softmax'))

### モデル構造の表示

In [ ]:
model.summary()

### モデルのコンパイル

先ほどのモデルと同じ様にモデルをコンパイルします。

In [ ]:
model.compile(loss='categorical_crossentropy', 
              optimizer='adam',
              metrics=['acc'])

### モデルの訓練

25エポック、モデルの訓練を行います。

In [ ]:
history = model.fit_generator(train_generator,
                              epochs=25,
                              steps_per_epoch=steps_per_epoch,
                              validation_data=validation_generator,
                              validation_steps=validation_steps)

### 学習経過のグラフ化

エポックごとの損失と正解率をグラフ化します。
先程のグラフとどの様に違うでしょうか？

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, label='Train Accuracy')
plt.plot(epochs, val_acc, label='Validation Accuracy')
plt.legend()
plt.show()

plt.plot(epochs, loss, label='Train Loss')
plt.plot(epochs, val_loss, label='Validation Loss')
plt.legend()
plt.show()

## バッチ正規化の導入

### モデルの修正

最初のモデルにBatchNormalization層を挿入してみましょう。

In [ ]:
model = keras.models.Sequential()

model.add(keras.layers.Conv2D(32, (3, 3), strides=(1, 1), input_shape=(128, 128, 3)))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Activation('relu'))
model.add(keras.layers.Conv2D(64, (3, 3), strides=(1, 1)))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Activation('relu'))
model.add(keras.layers.Conv2D(128, (3, 3), strides=(1, 1)))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Activation('relu'))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Activation('relu'))
model.add(keras.layers.Dense(2, activation='softmax'))

### モデル構造の表示

In [ ]:
model.summary()

### モデルのコンパイル

先ほどのモデルと同じ様にモデルをコンパイルします。

In [ ]:
model.compile(loss='categorical_crossentropy', 
              optimizer='adam',
              metrics=['acc'])

### モデルの訓練

30エポック、モデルの訓練を行います。

In [ ]:
history = model.fit_generator(train_generator,
                              epochs=30,
                              steps_per_epoch=steps_per_epoch,
                              validation_data=validation_generator,
                              validation_steps=validation_steps)

### 学習経過のグラフ化

エポックごとの損失と正解率をグラフ化します。
先程のグラフとどの様に違うでしょうか？

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, label='Train Accuracy')
plt.plot(epochs, val_acc, label='Validation Accuracy')
plt.legend()
plt.show()

plt.plot(epochs, loss, label='Train Loss')
plt.plot(epochs, val_loss, label='Validation Loss')
plt.legend()
plt.show()

## パラメータ正則化の導入

### モデルの修正

最初のモデルにパラメータ正則化の導入をしてみましょう。

In [ ]:
model = keras.models.Sequential()

model.add(keras.layers.Conv2D(32, (3, 3), 
                    strides=(1, 1), 
                    kernel_regularizer=keras.regularizers.l1_l2(0.0005, 0.001),
                    activation='relu', 
                    input_shape=(128, 128, 3)))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Conv2D(64, (3, 3), 
                    strides=(1, 1), 
                    kernel_regularizer=keras.regularizers.l1_l2(0.0005, 0.001),
                    activation='relu'))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Conv2D(128, (3, 3), 
                    strides=(1, 1), 
                    kernel_regularizer=keras.regularizers.l1_l2(0.0005, 0.001),
                    activation='relu'))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128, 
                    kernel_regularizer=keras.regularizers.l1_l2(0.0005, 0.001),
                    activation='relu'))
model.add(keras.layers.Dense(2, activation='softmax'))

### モデル構造の表示

In [ ]:
model.summary()

### モデルのコンパイル

先ほどのモデルと同じ様にモデルをコンパイルします。

In [ ]:
model.compile(loss='categorical_crossentropy', 
              optimizer='adam',
              metrics=['acc'])

### モデルの訓練

30エポック、モデルの訓練を行います。

In [ ]:
history = model.fit_generator(train_generator,
                              epochs=30,
                              steps_per_epoch=steps_per_epoch,
                              validation_data=validation_generator,
                              validation_steps=validation_steps)

### 学習経過のグラフ化

エポックごとの損失と正解率をグラフ化します。
先程のグラフとどの様に違うでしょうか？

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, label='Train Accuracy')
plt.plot(epochs, val_acc, label='Validation Accuracy')
plt.legend()
plt.show()

plt.plot(epochs, loss, label='Train Loss')
plt.plot(epochs, val_loss, label='Validation Loss')
plt.legend()
plt.show()

## モデルによる予測

下記のプログラムで、学習したモデルを使ってテスト用データの分類（予測）を行うことができます。  
予測結果がどのような形式なのかを確認してください。

In [ ]:
predictions = model.predict_generator(test_generator, steps=test_steps)

print(predictions)

## モデルの予測結果を見てみる

次のプログラムを実行すると、テスト用データの１バッチ分についてモデルの予測値と正解を画像つきで確認できます。  
どのような結果になったかを確認してください。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

code_to_class = {0: 'ラーメン', 1: 'うどん'}

x, y = next(test_generator)
y_true = np.argmax(y, axis=1)
y_prob = model.predict_on_batch(x)
y_pred = np.argmax(y_prob, axis=1)

for i in range(len(x)):
    plt.imshow(x[i])
    plt.show()
    print('推定：', code_to_class[y_pred[i]])
    print('正解：', code_to_class[y_true[i]])